In [1]:
# import and configs
! pip install web3 dotenv pandas

In [4]:
# importing the web3 object from web3.py

from web3 import Web3
import pandas as pd
from dotenv import load_dotenv
import os

# load env variable from .env file

load_dotenv()
GATEWAY = os.getenv("GATEWAY_URL")

print(GATEWAY)

https://mainnet.infura.io/v3/14b83f0d989a4d67bb9181a343654d0e


In [6]:
w3 = Web3(Web3.HTTPProvider(GATEWAY))

print(f'Is connected successfully= {w3.is_connected()}')

Is connected successfully= True


In [7]:
# function to change address to checksum

def turn_to_checksum(address: str) -> str:
    """converts address to its checksum format"""
    if not Web3.is_checksum_address(address):
        return Web3.to_checksum_address(address)
    return address

## Assignment
### create a function that convert block_number to timestamp(human readable)

In [10]:
def block_to_timestamp(block_number: int):
    """converts block number to timestamp"""
    block = w3.eth.get_block(block_number)
    timestamp = block.timestamp
    return pd.to_datetime(timestamp, unit='s')

    

In [11]:
block_to_timestamp(23248570)

Timestamp('2025-08-29 18:27:11')

### calculate average gas price of the last 10 blocks

In [ ]:

# calculate the average gas price of the last n blocks
def average_gas_price_last_blocks(n=10):
    """Calculate average gas price of the last n blocks"""
    latest_block = w3.eth.block_number
    gas_prices = []

    for block_num in range(latest_block, latest_block - n, -1):
        block = w3.eth.get_block(block_num, full_transactions=True)
        # Extract gas price from each transaction in the block
        for tx in block.transactions:
            gas_prices.append(tx.gasPrice)

    if not gas_prices:
        return None

    avg_gas_price = sum(gas_prices) / len(gas_prices)
    return w3.from_wei(avg_gas_price, 'gwei')  # convert to gwei for readability

print("Average gas price (last 10 blocks):", average_gas_price_last_blocks(10), "gwei")


Average gas price (last 10 blocks): 0.95155564401849567890167236328125 gwei


### Create a simple whale detector function

 Create a simple whale detector function

     - The function should:
         - Check if a wallet has more than 100 ETH
         - Check if a wallet has more than 1M USDC
         - If either is True, the wallet should be tagged a "Whale" (can simply print "Whale")
         - Optionally, can add more categories (goldfish, dolphin, small whale, large whale, etc.)
     - Test the function with your address or addresses you find on [etherscan.io](https://etherscan.io/token/0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48#balances)
     

In [ ]:
ERC20_ABI = [
    {
        "constant": True,
        "inputs": [{"name": "_owner", "type": "address"}],
        "name": "balanceOf",
        "outputs": [{"name": "balance", "type": "uint256"}],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "totalSupply",
        "outputs": [
            {
                "name": "",
                "type": "uint256"
            }
        ],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
    {
        "constant": True,
        "inputs": [],
        "name": "decimals",
        "outputs": [
            {
                "name": "",
                "type": "uint8"
            }
        ],
        "payable": False,
        "stateMutability": "view",
        "type": "function"
    },
]







In [ ]:
def detect_whale(address: str, tokens: dict) -> dict:
    """
    Detect if an address qualifies as a whale.
    
    Parameters:
        address (str): Ethereum wallet address.
        tokens (dict): Mapping of token name -> token contract address.
                       Example: {"USDC": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"}

    Returns:
        dict: balances and whale classification
    """
    result = {"address": address, "balances": {}, "tag": "Not Whale"}

    # --- ETH balance ---
    raw_ether_balance = w3.eth.get_balance(address)
    eth_balance = w3.from_wei(raw_ether_balance, 'ether')
    result["balances"]["ETH"] = eth_balance

    # --- Token balances ---
    for symbol, token_addr in tokens.items():
        token_contract = w3.eth.contract(address=token_addr, abi=ERC20_ABI)
        raw_balance = token_contract.functions.balanceOf(address).call()
        decimals = token_contract.functions.decimals().call()
        normalized_balance = raw_balance / (10 ** decimals)
        result["balances"][symbol] = normalized_balance

    # --- Whale detection rules ---
    if eth_balance > 100 or result["balances"].get("USDC", 0) > 1_000_000:
        result["tag"] = "Whale"

    if eth_balance > 1000 or result["balances"].get("USDC", 0) > 10_000_000:
        result["tag"] = "Mega Whale"
    elif eth_balance > 50 or result["balances"].get("USDC", 0) > 500_000:
        result["tag"] = "Dolphin"

    return result


# --- Example usage ---
tokens = {
    "USDC": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"  # Mainnet USDC
}

# Example call (replace with a real address)
# print(detect_whale("0x742d35Cc6634C0532925a3b844Bc454e4438f44e", tokens))


In [38]:
detect_whale("0x5B6122C109B78C6755486966148C1D70a50A47D7", tokens)

{'address': '0x5B6122C109B78C6755486966148C1D70a50A47D7',
 'balances': {'ETH': Decimal('113.727440597777858883'), 'USDC': 0.006797},
 'tag': 'Dolphin'}

In [39]:
def classify_addresses(addresses: list, tokens: dict) -> dict:
    """
    Classify multiple addresses into whale categories.
    
    Parameters:
        addresses (list): List of Ethereum addresses
        tokens (dict): Mapping of token name -> token contract address
    
    Returns:
        dict: Mapping {address: category_tag}
    """
    classification = {}

    for addr in addresses:
        result = detect_whale(addr, tokens)  # reuse the previous function
        classification[addr] = result["tag"]

    return classification


# --- Example usage ---
addresses = [
    "0x5B6122C109B78C6755486966148C1D70a50A47D7",  
    "0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97",  
]

tokens = {
    "USDC": "0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48"  # Mainnet USDC
}



In [40]:
classify_addresses(addresses, tokens)

{'0x5B6122C109B78C6755486966148C1D70a50A47D7': 'Dolphin',
 '0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97': 'Not Whale'}